## load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os

# Suppress logging warnings
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

In [3]:
df = pd.read_csv('./data/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
x = df.iloc[:, 1:].to_numpy()
y = df.iloc[:, 0].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [6]:
x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0

In [7]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
from sklearn.metrics import accuracy_score

2025-02-02 17:19:29.183659: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-02 17:19:29.202373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-02 17:19:29.217398: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-02 17:19:29.222547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 17:19:29.236646: I tensorflow/core/platform/cpu_feature_guar

In [8]:
class MyOAnn:
    def __init__(self, input_dim, output_dim, 
                 num_hidden_layers, neurons_per_layer,
                dropout_rate):
        
        self.model = Sequential()
        self.model.add(Input(shape=(input_dim, )))

        for _ in range(num_hidden_layers):
            self.model.add(Dense(128, activation='relu'))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(rate=dropout_rate))

        self.model.add(Dense(output_dim, activation='softmax'))


In [9]:
def objective(trial):
    
    #get hyperparameter values
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step=8)
    epochs = trial.suggest_int("epochs", 10, 50, step = 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64])
    optimizer_model = trial.suggest_categorical('optimizer_model', ['sgd', 'adam', 'rmsprop'])


    #model init
    input_dim = x_train.shape[1]
    output_dim = 10
    
    ann = MyOAnn(input_dim=input_dim, output_dim=output_dim,
                 num_hidden_layers=num_hidden_layers, neurons_per_layer= neurons_per_layer,
                dropout_rate=dropout_rate)

    # optimizer and loss
    if optimizer_model == 'adam':
        optimizer = Adam(learning_rate=learning_rate, weight_decay=weight_decay)
    elif optimizer_model == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = SGD(learning_rate=learning_rate, weight_decay=weight_decay)
        
    ann.model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])

    # training loop
    history = ann.model.fit(x_train_scaled, y_train, epochs= epochs, 
                            validation_split=0.2, batch_size=batch_size)

    # evaluation loop
    y_prob = ann.model.predict(x_test)
    y_pred = y_prob.argmax(axis=1)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [10]:
import optuna
study = optuna.create_study(direction='maximize')

[I 2025-02-02 17:19:32,257] A new study created in memory with name: no-name-5e8236e7-06ee-4bf9-83d8-b15a0fe8ea56


In [ ]:
study.optimize(objective, n_trials=10)

I0000 00:00:1738541973.239012 1410778 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738541973.281100 1410778 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738541973.284639 1410778 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738541973.289949 1410778 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/40


I0000 00:00:1738541975.418093 1411013 service.cc:146] XLA service 0x75067c0146b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738541975.418141 1411013 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-02-02 17:19:35.479532: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-02 17:19:35.601796: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300


 37/240 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3982 - loss: 2.0050  

I0000 00:00:1738541977.932003 1411013 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.1748 - loss: 2.7907 - val_accuracy: 0.3677 - val_loss: 1.8961
Epoch 2/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3358 - loss: 2.0645 - val_accuracy: 0.4740 - val_loss: 1.5485
Epoch 3/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4304 - loss: 1.7245 - val_accuracy: 0.5500 - val_loss: 1.3325
Epoch 4/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4901 - loss: 1.4940 - val_accuracy: 0.5927 - val_loss: 1.1944
Epoch 5/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5178 - loss: 1.4212 - val_accuracy: 0.6354 - val_loss: 1.1003
Epoch 6/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5481 - loss: 1.3375 - val_accuracy: 0.6677 - val_loss: 1.0293
Epoch 7/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5907 - loss: 1.2678 - val_accuracy: 0.6854 - val_loss: 0.9828
Epoch 8/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5839 - loss: 1.2171 - val_accuracy: 0.

[I 2025-02-02 17:27:57,855] Trial 6 finished with value: 0.5766666666666667 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 128, 'epochs': 50, 'learning_rate': 9.1632198663012e-05, 'dropout_rate': 0.1, 'weight_decay': 0.0005482765848427803, 'batch_size': 8, 'optimizer_model': 'sgd'}. Best is trial 3 with value: 0.7508333333333334.


Epoch 1/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.3677 - loss: 2.0423 - val_accuracy: 0.7083 - val_loss: 0.8056
Epoch 2/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5371 - loss: 1.2573 - val_accuracy: 0.6979 - val_loss: 0.8166
Epoch 3/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5632 - loss: 1.2230 - val_accuracy: 0.7510 - val_loss: 0.6981
Epoch 4/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6059 - loss: 1.0920 - val_accuracy: 0.7406 - val_loss: 0.7430
Epoch 5/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6105 - loss: 1.0834 - val_accuracy: 0.7042 - val_loss: 0.7236
Epoch 6/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6420 - loss: 1.0376 - val_accuracy: 0.7500 - val_loss: 0.6969
Epoch 7/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6318 - loss: 1.0380 - val_accuracy: 0.7312 - val_loss: 0.6583
Epoch 8/50
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6284 - loss: 1.0303 - val_accuracy: 0.

[I 2025-02-02 17:29:54,793] Trial 7 finished with value: 0.6133333333333333 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 8, 'epochs': 50, 'learning_rate': 0.004853693504902021, 'dropout_rate': 0.5, 'weight_decay': 0.0006052665432249888, 'batch_size': 8, 'optimizer_model': 'adam'}. Best is trial 3 with value: 0.7508333333333334.


Epoch 1/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.1754 - loss: 2.6342 - val_accuracy: 0.3646 - val_loss: 1.8984
Epoch 2/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3103 - loss: 2.0702 - val_accuracy: 0.4698 - val_loss: 1.6370
Epoch 3/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3703 - loss: 1.8306 - val_accuracy: 0.5333 - val_loss: 1.3989
Epoch 4/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4358 - loss: 1.6487 - val_accuracy: 0.5667 - val_loss: 1.2638
Epoch 5/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4724 - loss: 1.5708 - val_accuracy: 0.6073 - val_loss: 1.1397
Epoch 6/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4795 - loss: 1.4965 - val_accuracy: 0.6375 - val_loss: 1.0743
Epoch 7/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5231 - loss: 1.4124 - val_accuracy: 0.6646 - val_loss: 1.0146
Epoch 8/40
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5361 - loss: 1.3525 - val_accuracy: 0

In [27]:
study.best_value

0.83